In [80]:
# UT-TOR-DATA-PT-01-2020-U-C Group Project 2
# Scrape https://acharts.co/canada_singles_top_100

## Imports

In [81]:
# SQLite Database related imports
import sqlalchemy
from sqlalchemy.engine import Engine
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, event, inspect

In [82]:
# Scraping related imports
from bs4 import BeautifulSoup
from splinter import Browser

In [83]:
import time

## Initializations

### DB access initialization

In [84]:
# Turn on PRAGMA foreign_keys to enforce foregn key constraints (it is disabled by default in SQLite)
@event.listens_for(Engine, "connect")
def set_sqlite_pragma(dbapi_connection, connection_record):
    cursor = dbapi_connection.cursor()
    cursor.execute("PRAGMA foreign_keys=ON")
    cursor.close()

# Create engine to access the database
engine = create_engine("sqlite:///../data/CanadaTop100.sqlite")

# Reflect an existing database into a new model
AutomapBase = automap_base()

# Reflect the tables
AutomapBase.prepare(engine, reflect=True)

# Save references to each table
Artist = AutomapBase.classes.Artist
Chart = AutomapBase.classes.Chart
Song = AutomapBase.classes.Song
Performed_by = AutomapBase.classes.Performed_by

# Create our session (link) from Python to the DB
session = Session(engine)

# Debug output
#AutomapBase.classes.items()
#list(inspect(Artist).columns)
#list(inspect(Chart).columns)
#list(inspect(Song).columns)
#list(inspect(Performed_by).columns)

### Scraper initialization

In [85]:
base_url = 'https://acharts.co/canada_singles_top_100'
archive_url = base_url + '#archive'
browser = Browser('chrome', executable_path='chromedriver.exe', headless=False)

## Helper functions

In [86]:
# Checks if this week was already saved into the database
# Returnes true if the year+week data is in the Chart table, false otherwise
def in_database(year, week):
    count = session.query(Chart).filter_by(year=year, week=week).count()
    return count > 0

In [87]:
# Checks if the song is in the Song table.
# Returnes song_id or Null if the song isn't in the table
def check_song(song, performed_by):
    song_id = session.query(Song.id).filter_by(song_name=song, performed_by=performed_by).scalar()
    #print(f"Checking: song='{song}', performed_by='{performed_by}', id={song_id}")
    return song_id

In [88]:
# Adds new song to the Song table
# Returns song_id
def insert_song(song, performed_by):
    session.add(Song(song_name=song, performed_by=performed_by))
    session.commit()
    song_id = session.query(Song.id).filter_by(song_name=song, performed_by=performed_by).scalar()
    #print(f"New song: song='{song}', performed_by='{performed_by}', id={song_id}")
    return song_id

In [89]:
# Checks all artists in the Artist table, inserts new artists.
# Returnes a list of artist ids
def check_artists(artists):
    artist_ids = []
    
    for artist in artists:
        artist_id = session.query(Artist.id).filter_by(name=artist).scalar()
        #print(f"Existing artist: name={artist}, id={artist_id}")
        
        if not artist_id:
            session.add(Artist(name=artist))
            session.commit()
            artist_id = session.query(Artist.id).filter_by(name=artist).scalar()
            #print(f"New artist: name={artist}, id={artist_id}")

        artist_ids.append(artist_id)

    #print(f"List of artist ids: {artist_ids}")
    return artist_ids

In [90]:
# Adds entries to the Performed_by table for a new song
def insert_performers(song_id, artists):
    artist_ids = check_artists(artists)
    rows = []
    for idx, artist_id in enumerate(artist_ids):
        rows.append(Performed_by(song_id=song_id, artist_id=artist_id, order=idx))
    session.add_all(rows)
    session.commit()

In [91]:
# Adds a song in the Chart table
def update_chart(year, week, position, song_id):
    session.add(Chart(year=year, week=week, position=position, song_id=song_id))
    session.commit()

In [92]:
# Processes one row of a chart table
# Parameters: year, week and row DOM node
def add_chart_row(year, week, row):
    # Parsing the row with song information
    pos_node = row.find('td', class_='cNum')
    position = pos_node.find('span', itemprop='position').string

    hit_node = row.find('td', class_='cPrinciple')
    song = hit_node.find('span', itemprop='name').string

    performed_node = hit_node.br.find_next('span')
    performed_by = ' '.join(performed_node.stripped_strings)

    artists = []
    for artist in performed_node.find_all('span', itemprop='name'):
        artists.append(artist.string)

    # Ad Hoc corrections - working around data bugs
    # Bug #1: 2018/46, position 88
    if len(artists) == 3 and artists[1] == 'Jack' and artists[2] == 'Jack':
        artists = artists[:2]
        artists[1] = 'Jack & Jack'
        
    #print(f"year={year}, week={week}, position={position}, song='{song}', performed_by='{performed_by}', artists={artists}")
    
    # Saving the row in the database
    song_id = check_song(song, performed_by)
    if not song_id:
        song_id = insert_song(song, performed_by)
        insert_performers(song_id, artists)

    update_chart(year, week, position, song_id)

In [93]:
# Scrape one week chart
# Parameters: year, week
def scrape_week(year, week):
    week_url = f"{base_url}/{year}/{week}"
    browser.visit(week_url)
    soup = BeautifulSoup(browser.html, 'lxml')

    chart = soup.find('table', id='ChartTable').tbody
    if not chart:
        print("Didn't find 'table' element on the chart page.\nDEBUG:")
        print(soup)
        return False

    rows = chart.find_all('tr')
    if rows is None: # Data bug #2: week 53 2009, 96 songs
        print(f"Chart table rows not found.\nDEBUG:")
        print(chart)
        return False
        
    for row in rows:
        add_chart_row(year, week, row)

    return True

## Scraping

In [94]:
# Get the list of all weeks and years
browser.visit(archive_url)
soup = BeautifulSoup(browser.html, 'lxml')
select_node = soup.find('select', id='SelectWeek')
option_nodes = select_node.find_all('option')
weeks = [option['value'] for option in option_nodes]
weeks.reverse() # reversing the list to start from the most recent charts and go back to the oldest

In [ ]:
# Iterate through the week strings (in the form "year/week") and scraping all of them one by one
for week_str in weeks:
    year, week = week_str.split('/')
    if not in_database(year, week):
        print("Scraping week " + week_str)
        time.sleep(3)
        if not scrape_week(year, week):
            print("Something went wrong. Stopping...")
            break
    else:
        print("Skipping week " + week_str + " - already in the database")

In [96]:
browser.quit()